In [2]:
import json
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# Normalize json annotation to Solera classes

In [5]:
annotation_file = Path('./dataset/cogito_output_merged.json')

with open(annotation_file, 'r') as f:
    annotations = json.load(f)
    
solera_mapping = {'back_side' : 'segment_back_side',
 'back_window': 'segment_back_window',
 'bumper_back': 'segment_back_bumper',
 'bumper_front': 'segment_front_bumper',
 'door_front_left': 'segment_front_door_left',
 'door_front_right': 'segment_front_door_right',
 'door_pillar_left': 'segment_door_pillar_left',
 'door_pillar_right': 'segment_door_pillar_right',
 'door_rear_left': 'segment_rear_door_left',
 'door_rear_right': 'segment_rear_door_right',
 'door_sill_left': 'segment_door_sill_left',
 'door_sill_right': 'segment_door_sill_right',
 'door_window_front_left': 'segment_front_door_window_left',
 'door_window_front_right': 'segment_front_door_window_right',
 'door_window_rear_left' : 'segment_rear_door_left',
 'door_window_rear_right': 'segment_rear_door_right',
 'fender_back_left': 'segment_back_fender_left',
 'fender_back_right': 'segment_back_fender_right',
 'fender_front_left': 'segment_front_fender_left',
 'fender_front_right': 'segment_front_fender_right',
 'fog_light_back_left': 'segment_fog_light_left', # aggregate
 'fog_light_back_right': 'segment_fog_light_right',
 'fog_light_front_left': 'segment_fog_light_left',
 'fog_light_front_right': 'segment_fog_light_right',
 'handle_front_left': 'segment_car_handle_front',
 'handle_front_right': 'segment_car_handle_front',
 'handle_rear_left': 'segment_car_handle_back',
 'handle_rear_right': 'segment_car_handle_back',
 'headlight_left': 'segment_front_headlight_left',
 'headlight_right': 'segment_front_headlight_right',
 'hood': 'segment_front_side',
 'mirror_left': 'segment_left_mirror',
 'mirror_right': 'segment_left_right',
 'plate_back': 'segment_back_plate',
 'plate_front': 'segment_front_plate',
 'radiator_grill': 'segment_radiator_grill',
 'roof': 'segment_roofside',
 'tail_light_left': 'segment_taillight_left',
 'tail_light_right': 'segment_taillight_right',
 'trunk': 'segment_back_side',
 'wheels_back_left': 'segment_wheels_back_left',
 'wheels_back_right': 'segment_wheels_back_right',
 'wheels_front_left': 'segment_wheels_front_left',
 'wheels_front_right': 'segment_wheels_front_right',
 'windscreen': 'segment_windscreen'}

In [6]:
id_name_old = [ (a['id'], a['name']) for a in annotations['categories'] ]
df = pd.DataFrame(id_name_old, columns = ['old_id', 'old_name'])
df.head()

,old_id,old_name
0,0,back_side
1,1,back-side
2,2,back-window
3,3,bumper_back
4,4,bumper_front


In [7]:
df = df[df['old_name'] != '']
df['new_name'] = df['old_name'].apply(lambda x: x.replace('-','_').lower())
df['solera_name'] = df['new_name'].apply(lambda x: solera_mapping[x] if x in solera_mapping else x)

In [8]:
df.head()

,old_id,old_name,new_name,solera_name
0,0,back_side,back_side,segment_back_side
1,1,back-side,back_side,segment_back_side
2,2,back-window,back_window,segment_back_window
3,3,bumper_back,bumper_back,segment_back_bumper
4,4,bumper_front,bumper_front,segment_front_bumper


In [9]:
df.groupby('new_name').count().sort_values(by='old_name', ascending=False).head()

,old_id,old_name,solera_name
new_name,,,
back_side,2,2,2
handle_front_left,1,1,1
handle_rear_left,1,1,1
handle_rear_right,1,1,1
headlight_left,1,1,1


In [10]:
new_names = df['solera_name'].unique().tolist()
new_names = [ (idx, name) for idx, name in enumerate(sorted(new_names)) ]

df_new = pd.DataFrame(new_names, columns = ['new_id', 'solera_name'])

In [11]:
new_names

[(0, 'dent'),
 (1, 'scratch'),
 (2, 'segment_back_bumper'),
 (3, 'segment_back_fender_left'),
 (4, 'segment_back_fender_right'),
 (5, 'segment_back_plate'),
 (6, 'segment_back_side'),
 (7, 'segment_back_window'),
 (8, 'segment_car_handle_back'),
 (9, 'segment_car_handle_front'),
 (10, 'segment_door_pillar_left'),
 (11, 'segment_door_pillar_right'),
 (12, 'segment_door_sill_left'),
 (13, 'segment_door_sill_right'),
 (14, 'segment_fog_light_left'),
 (15, 'segment_fog_light_right'),
 (16, 'segment_front_bumper'),
 (17, 'segment_front_door_left'),
 (18, 'segment_front_door_right'),
 (19, 'segment_front_door_window_left'),
 (20, 'segment_front_door_window_right'),
 (21, 'segment_front_fender_left'),
 (22, 'segment_front_fender_right'),
 (23, 'segment_front_headlight_left'),
 (24, 'segment_front_headlight_right'),
 (25, 'segment_front_plate'),
 (26, 'segment_front_side'),
 (27, 'segment_left_mirror'),
 (28, 'segment_left_right'),
 (29, 'segment_radiator_grill'),
 (30, 'segment_rear_door_left

In [12]:
map_df = df.merge(df_new, on='solera_name', how='inner')
map_df.head(10)

,old_id,old_name,new_name,solera_name,new_id
0,0,back_side,back_side,segment_back_side,6
1,1,back-side,back_side,segment_back_side,6
2,44,trunk,trunk,segment_back_side,6
3,2,back-window,back_window,segment_back_window,7
4,3,bumper_back,bumper_back,segment_back_bumper,2
5,4,bumper_front,bumper_front,segment_front_bumper,16
6,5,dent,dent,dent,0
7,6,door_front_left,door_front_left,segment_front_door_left,17
8,7,door_front_right,door_front_right,segment_front_door_right,18
9,8,door-pillar_left,door_pillar_left,segment_door_pillar_left,10


In [13]:
id_map = dict(zip(map_df['old_id'],map_df['new_id']))
name_map = dict(zip(map_df['old_name'],map_df['solera_name']))

In [14]:
name_map

{'back_side': 'segment_back_side',
 'back-side': 'segment_back_side',
 'trunk': 'segment_back_side',
 'back-window': 'segment_back_window',
 'bumper_back': 'segment_back_bumper',
 'bumper_front': 'segment_front_bumper',
 'dent': 'dent',
 'door_front_left': 'segment_front_door_left',
 'door_front_right': 'segment_front_door_right',
 'door-pillar_left': 'segment_door_pillar_left',
 'door-pillar_right': 'segment_door_pillar_right',
 'door_rear_left': 'segment_rear_door_left',
 'door-window_rear_left': 'segment_rear_door_left',
 'door_rear_right': 'segment_rear_door_right',
 'door-window_rear_right': 'segment_rear_door_right',
 'door-sill_left': 'segment_door_sill_left',
 'door-sill_right': 'segment_door_sill_right',
 'door-window_front_left': 'segment_front_door_window_left',
 'door-window_front_right': 'segment_front_door_window_right',
 'fender_back_left': 'segment_back_fender_left',
 'fender_back_right': 'segment_back_fender_right',
 'fender_front_left': 'segment_front_fender_left',
 '

In [15]:
import copy
new_annotations = copy.copy(annotations)

In [16]:
for entry in new_annotations['categories']:
    if entry['id'] in id_map.keys():
        entry['id'] = id_map[entry['id']]
        entry['name'] = name_map[entry['name']]
    else: 
        del entry

In [17]:
for entry in new_annotations['annotations']:
    if entry['category_id'] in id_map.keys():
        entry['category_id'] = id_map[entry['category_id']]
    else: 
        del entry

In [19]:
with open('./dataset/cogito_output_merged_solera.json', 'w') as f:
    json.dump(new_annotations, f)